In [21]:
#coding=utf-8
import tushare as ts
import pandas as pd
import talib
import time
import datetime
from opendatatools import swindex

ts.set_token('bc406c9fab768c6c699ef450685daed298c5abf2dbdba8da4e810cab')
pro = ts.pro_api()


In [2]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# 获取指数(综合指数、规模指数、一级行业指数、二级行业指数、策略指数、成长指数、价值指数、主题指数)K线数据
# 综合指数，例如：sh.000001 上证指数，sz.399106 深证综指 等；
# 规模指数，例如：sh.000016 上证50，sh.000300 沪深300，sh.000905 中证500，sz.399001 深证成指等；
# 一级行业指数，例如：sh.000037 上证医药，sz.399433 国证交运 等；
# 二级行业指数，例如：sh.000952 300地产，sz.399951 300银行 等；
# 策略指数，例如：sh.000050 50等权，sh.000982 500等权 等；
# 成长指数，例如：sz.399376 小盘成长 等；
# 价值指数，例如：sh.000029 180价值 等；
# 主题指数，例如：sh.000015 红利指数，sh.000063 上证周期 等；

# 详细指标参数，参见“历史行情指标参数”章节
rs = bs.query_history_k_data_plus("000906.sh",
    "date,code,open,high,low,close,preclose,volume,amount,pctChg",
    start_date='2014-01-01', end_date='2020-10-23', frequency="d")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

# 打印结果集
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)
result.to_csv("C:/quant/index/history_Index_k_data.csv", index=False)# 登出系统
bs.logout()

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
logout success!


In [3]:
#因为baostock出数据慢，手动输入当日数据后也能算MA

result=pd.read_csv("C:/quant/index/history_Index_k_data.csv")# 登出系统

ma5=talib.SMA(result.close,timeperiod=5)
ma10=talib.SMA(result.close,timeperiod=10)
ma20=talib.SMA(result.close,timeperiod=20)
ma30=talib.SMA(result.close,timeperiod=30)
ma60=talib.SMA(result.close,timeperiod=60)
ma120=talib.SMA(result.close,timeperiod=120)

ema5=talib.SMA(result.close,timeperiod=5)
ema10=talib.SMA(result.close,timeperiod=10)
ema20=talib.SMA(result.close,timeperiod=20)
ema30=talib.SMA(result.close,timeperiod=30)
ema60=talib.SMA(result.close,timeperiod=60)
ema120=talib.SMA(result.close,timeperiod=120)


result['MA5']=ma5
result['MA10']=ma10
result['MA20']=ma20
result['MA30']=ma30
result['MA60']=ma60
result['MA120']=ma120
result['EMA5']=ema5
result['EMA10']=ema10
result['EMA20']=ema20
result['EMA30']=ema30
result['EMA60']=ema60
result['EMA120']=ema120


# 结果集输出到csv文件
result.to_csv("C:/quant/index/history_Index_k_data.csv", index=False)
print(result)

# 登出系统
bs.logout()

            date       code       open       high        low      close  \
0     2014-01-02  sh.000906  2582.9100  2586.9400  2570.9500  2586.0200   
1     2014-01-03  sh.000906  2576.6100  2579.2900  2544.5900  2554.8000   
2     2014-01-06  sh.000906  2549.4700  2549.4700  2487.0700  2493.4000   
3     2014-01-07  sh.000906  2478.0200  2504.6600  2473.2700  2496.8600   
4     2014-01-08  sh.000906  2499.3500  2523.1300  2488.0000  2500.8800   
...          ...        ...        ...        ...        ...        ...   
1653  2020-10-19  sh.000906  5089.7820  5112.9990  5005.9923  5014.7910   
1654  2020-10-20  sh.000906  5009.8481  5057.4922  4996.0496  5057.4922   
1655  2020-10-21  sh.000906  5064.6191  5064.6191  5011.8987  5043.1498   
1656  2020-10-22  sh.000906  5029.8992  5039.0519  4971.4497  5025.0332   
1657  2020-10-23  sh.000906  5022.0431  5049.3727  4954.5464  4957.6898   

       preclose       volume        amount    pctChg  ...         MA20  \
0     2589.3500   7599455

In [22]:

pool = pro.stock_basic()
pool.head(5)
writer = pd.ExcelWriter('C:/quant/list/list.xlsx') 
pool.to_excel(writer,'Sheet1')
writer.save()
writer.close()


In [23]:
#从申银模块获取行业列表
SWf, msg = swindex.get_index_list()
S1=SWf.loc[SWf['section_name']=='一级行业']
#lc.loc[lc["grade"] == "B"].head()


In [24]:
#从申银端口通过刚才过滤的一级行业列表获取数据
allindex = pd.DataFrame() 

for i in S1.index_code:
    time.sleep(0.75)
    X=str(i)
    df2, msg = swindex.get_index_daily(X,start_date=20200101,end_date=20201029)
    #df2.to_csv(r'C:/quant/index/2020/SW/'+X+'.csv',index=False,encoding='utf-8-sig') 

    allindex = allindex.append(df2)
allindex = allindex.sort_values(by=['index_code','date'],axis=0,ascending=[True,True])

   


http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('801010') and BargainDate >= '20200101' and BargainDate <= '20201029' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('801020') and BargainDate >= '20200101' and BargainDate <= '20201029' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('801030') and BargainDate >= '20200101' and BargainDate <= '20201029' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('801040') and BargainDate >= '20200101' and BargainDate <= '20201029' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('801050') and BargainDate >= '20200101' and BargainDate <= '20201029' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('801080') and BargainDate >= '20200101' and BargainDate <= '20201029' 
http://www.swsindex.com/excel2.aspx?ctable=swindexhistory&where= swindexcode in ('801110

In [25]:
#申银源数据格式不对，转换格式
allindex['open']=allindex['open'].astype("float")
allindex['high']=allindex['high'].astype("float")
allindex['low']=allindex['low'].astype("float")
allindex['close']=allindex['close'].astype("float")
allindex['vol']=allindex['vol'].astype("float")
allindex['amount']=allindex['amount'].astype("float")
allindex['change_pct']=allindex['change_pct'].astype("float")
allindex.isnull().sum()


index_code    0
index_name    0
date          0
open          0
high          0
low           0
close         0
vol           0
amount        0
change_pct    0
dtype: int64

In [26]:
#计算申银行业指数MA，MACD
from scipy.stats import rankdata
swsf=allindex[allindex['index_code'].notna()]

close=swsf.close.values
swsf.date=swsf.date.values

diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
SMA5=talib.SMA(close,timeperiod=5)
SMA10=talib.SMA(close,timeperiod=10)
SMA20=talib.SMA(close,timeperiod=20)
SMA60=talib.SMA(close,timeperiod=60)
SMA120=talib.SMA(close,timeperiod=120)
SMA350=talib.SMA(close,timeperiod=350)

EMA5=talib.EMA(close,timeperiod=5)
EMA10=talib.EMA(close,timeperiod=10)
EMA20=talib.EMA(close,timeperiod=20)
EMA60=talib.EMA(close,timeperiod=60)
EMA120=talib.EMA(close,timeperiod=120)

swsf['diff']=diff
swsf['dea']=dea
swsf['MACD']=my_macd
swsf['SMA5']=SMA5
swsf['SMA10']=SMA10
swsf['SMA20']=SMA20
swsf['SMA60']=SMA60
swsf['SMA120']=SMA120
swsf['SMA350']=SMA350

swsf['EMA5']=EMA5
swsf['EMA10']=EMA10
swsf['EMA60']=EMA60
swsf['EMA120']=EMA120

swsf['EMA20']=EMA20
swsf['SMACD-K']=(diff-dea)/dea
swsf['C/S']=(close/SMA20-1)*100
swsf['M/L']=(SMA60/SMA120-1)*100
swsf['S/M']=(SMA20/SMA60-1)*100
swsf['BreakSignal-20D']=close/EMA20-1
swsf.reset_index(drop=True, inplace=True)

swsf['dailyamount']=swsf.groupby('date')['amount'].sum()
swsf['indexamount']=swsf.amount/swsf.dailyamount


swsf.loc[swsf['index_code'] == swsf['index_code'].shift(1),'Chg1D']= (swsf.close/swsf.close.shift(1)-1)*100
swsf.loc[swsf['index_code'] == swsf['index_code'].shift(4),'Chg5D']= (swsf.close/swsf.close.shift(4)-1)*100
swsf.loc[swsf['index_code'] == swsf['index_code'].shift(19),'Chg20D']= (swsf.close/swsf.close.shift(19)-1)*100

swsf.fillna(0,inplace=True) #去除null
swsf.loc[(swsf['close']>swsf['EMA20']) & (swsf['close']>swsf['SMA20']) & (swsf['SMA20']>swsf['SMA20'].shift(19)),'K1']=1 #破线拐头
swsf.loc[(swsf['EMA20']>swsf['EMA60']) & (swsf['close']>swsf['EMA20'].shift(19)),'K2']=1 #双均线交叉
swsf.loc[(swsf['EMA20']>swsf['SMA60']) & (swsf['close']>swsf['EMA20'].shift(19)),'K3']=1 
swsf.loc[(swsf['SMA20']>swsf['EMA60']) & (swsf['close']>swsf['SMA20'].shift(19)),'K4']=1 
swsf.loc[(swsf['SMA20']>swsf['SMA60']) & (swsf['close']>swsf['SMA20'].shift(19)),'K5']=1 

swsf.loc[(swsf['close']<swsf['EMA20']) & (swsf['close']<swsf['SMA20']) & (swsf['SMA20']<swsf['SMA20'].shift(19)),'K1']=-1 #空头方向破线拐头
swsf.loc[(swsf['EMA20']<swsf['EMA60']) & (swsf['close']<swsf['EMA20'].shift(19)),'K2']=-1 #双均线交叉
swsf.loc[(swsf['EMA20']<swsf['SMA60']) & (swsf['close']<swsf['EMA20'].shift(19)),'K3']=-1 
swsf.loc[(swsf['SMA20']<swsf['EMA60']) & (swsf['close']<swsf['SMA20'].shift(19)),'K4']=-1 
swsf.loc[(swsf['SMA20']<swsf['SMA60']) & (swsf['close']<swsf['SMA20'].shift(19)),'K5']=-1 
swsf.fillna(0,inplace=True) #去除null
swsf['Indextrendscore-L']=(swsf.K1+swsf.K2+swsf.K3+swsf.K4+swsf.K5)

swsf.head(10)
#swsf.to_csv('C:/quant/index/SWoutput.csv',index=False,encoding='utf-8-sig') 

,index_code,index_name,date,open,high,low,close,vol,amount,change_pct,...,indexamount,Chg1D,Chg5D,Chg20D,K1,K2,K3,K4,K5,Indextrendscore-L
0,801010,农林牧渔,2020-01-02,3347.27,3408.34,3345.43,3394.96,16.96,210.61,2.63,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,801010,农林牧渔,2020-01-03,3420.59,3436.81,3392.78,3425.51,18.30,195.75,0.90,...,0.0,0.899863,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,801010,农林牧渔,2020-01-06,3426.80,3509.56,3426.74,3494.01,20.19,205.22,2.00,...,0.0,1.999702,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,801010,农林牧渔,2020-01-07,3531.88,3647.21,3530.23,3624.72,23.28,306.57,3.74,...,0.0,3.740974,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,801010,农林牧渔,2020-01-08,3634.48,3642.81,3559.93,3559.93,22.49,275.67,-1.79,...,0.0,-1.787448,4.859262,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,801010,农林牧渔,2020-01-09,3582.81,3606.12,3547.75,3566.77,22.31,237.45,0.19,...,0.0,0.192139,4.123766,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,801010,农林牧渔,2020-01-10,3565.13,3565.14,3487.57,3493.82,18.88,207.00,-2.05,...,0.0,-2.045268,-0.005438,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,801010,农林牧渔,2020-01-13,3480.89,3517.73,3470.31,3517.73,16.26,180.90,0.68,...,0.0,0.684351,-2.951676,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,801010,农林牧渔,2020-01-14,3515.16,3546.22,3494.20,3520.44,19.85,213.92,0.08,...,0.0,0.077038,-1.109291,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,801010,农林牧渔,2020-01-15,3538.07,3538.08,3458.79,3466.91,14.60,169.30,-1.52,...,0.0,-1.520549,-2.799732,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:

#从TUSHARE获取申万一级行业列表及其对应股票
Sdf1 = pro.index_classify(level='L1', src='SW')
Sdf1.to_csv('C:/quant/swsindex/L1.csv',index=False,encoding='utf-8-sig') 
#获取申万二级行业列表
Sdf2 = pro.index_classify(level='L2', src='SW')
Sdf2.to_csv('C:/quant/swsindex/L2.csv',index=False,encoding='utf-8-sig')

#获取申万三级级行业列表
Sdf3 = pro.index_classify(level='L3', src='SW')
Sdf3.to_csv('C:/quant/swsindex/L3.csv',index=False,encoding='utf-8-sig') 

#Sdf3.head(10)
#SW1=pd.merge(Sdf1,Sdf2,how='right',on=['index_code','index_code'])
#SW2=pd.merge(SW1,Sdf2,how='right',on=['index_code','index_code'])


In [28]:

#从TUSHARE端口获取成分股
ISindex = pd.DataFrame() 

for i in Sdf1.index_code:
    time.sleep(0.7)
    Tf=pro.index_member(index_code=i)    
    ISindex = ISindex.append(Tf)

ISindex.to_csv(r'C:/quant/swsindex/total.csv') 
ISindex.rename(columns={'con_code':'ts_code'}, inplace = True)
ISindex=pd.merge(ISindex,Sdf1,how='left',on='index_code')
ISindex=ISindex.drop(["in_date",'out_date','level'],axis=1)

ISindex
    


,index_code,ts_code,industry_name
0,801020.SI,600348.SH,采掘
1,801020.SI,600121.SH,采掘
2,801020.SI,600157.SH,采掘
3,801020.SI,600792.SH,采掘
4,801020.SI,600123.SH,采掘
...,...,...,...
3730,801230.SI,600175.SH,综合
3731,801230.SI,600149.SH,综合
3732,801230.SI,600234.SH,综合
3733,801230.SI,300688.SZ,综合


In [29]:
latestday=max(swsf.date)
ETF=swsf[swsf.date==latestday]
ETF=ETF[['index_code','K1','Indextrendscore-L']]
ISindex['index_code']=ISindex['index_code'].str.split('.', 2).str[0]

ISindex2 = pd.merge(ISindex, ETF, how='left',on='index_code') #分段信息
ISindex2

,index_code,ts_code,industry_name,K1,Indextrendscore-L
0,801020,600348.SH,采掘,-1.0,-5.0
1,801020,600121.SH,采掘,-1.0,-5.0
2,801020,600157.SH,采掘,-1.0,-5.0
3,801020,600792.SH,采掘,-1.0,-5.0
4,801020,600123.SH,采掘,-1.0,-5.0
...,...,...,...,...,...
3730,801230,600175.SH,综合,0.0,-4.0
3731,801230,600149.SH,综合,0.0,-4.0
3732,801230,600234.SH,综合,0.0,-4.0
3733,801230,300688.SZ,综合,0.0,-4.0


In [74]:

#从tushare下载日数据

for i in range(20200901,20200932):
    time.sleep(0.7)
    print (i) 
    x=str(i)
    df = pro.daily(trade_date=x) #个股数据
    Adf = pro.query('adj_factor', trade_date=x) #复权因子
    Adf.rename(columns={'trade_date':'trade_date_x'}, inplace = True)
    Adf.adj_factor.fillna(0,inplace=True)


    Mdf = pro.moneyflow(trade_date=x) #个股资金流向，出韭菜指数
    X1=pd.merge(df,Mdf,on='ts_code')
    X1=pd.merge(X1,Adf,on=['ts_code','trade_date_x'])

    mf = pro.margin(trade_date=x)#融资融券数据
    GCdf = pro.repo_daily(ts_code='204001.sh',trade_date=x) # 逆回购数据
    #HSdf = pro.repo_daily(ts_code='000906.sh',trade_date=x) # 逆回购数据

    #X2=pd.merge(mf,GCdf,on='trade_date')

    #df2 = pro.index_dailybasic(start_date=x,fields='ts_code,trade_date,turnover_rate,pe,pe_ttm')
    #pe = df2['pe_ttm']


    writer1 = X1.to_csv('C:/quant/daydate/2020/'+x + '.csv') 
    writer2 = GCdf.to_csv('C:/quant/GCdata/2020/'+x + '.csv') 
    writer3 = mf.to_csv('C:/quant/financing/2020/'+x + '.csv') #要晚一天出，所以不能合并
    #writer4 = HSdf.to_csv('C:/quant/index/2020/'+x + '.csv') #要晚一天出，所以不能合并


   # df.to_csv(writer)
  #  writer.save()
   # writer.close()
    


20200901
20200902
20200903
20200904
20200905
20200906
20200907
20200908
20200909
20200910
20200911
20200912
20200913
20200914
20200915
20200916
20200917
20200918
20200919
20200920
20200921
20200922
20200923
20200924
20200925
20200926
20200927
20200928
20200929
20200930
20200931


In [75]:
import pandas as pd
import datetime
import numpy as np
from os import walk
for root,dirs,files in walk(r'C:/quant/daydate/2020/',topdown=False):
    num = len(files)
    alldata = pd.DataFrame() 
for i in range(num):
    newdata = pd.read_csv(r'C:/quant/daydate/2020/%s'%files[i])
    
    alldata = alldata.append(newdata)
alldata = alldata.sort_values(by=['ts_code','trade_date_x'],axis=0,ascending=[True,True])
alldata.reset_index(drop=True, inplace=True)


In [76]:
latestday=alldata.trade_date_x.max()
alldata.adj_factor.fillna(1,inplace=True)
alldata.loc[alldata['trade_date_x']==latestday,'now_adj']=alldata['adj_factor'] # 最新复权因子
alldata.now_adj=alldata.now_adj.fillna(method='bfill')

#alldata.loc[alldata['ts_code']==alldata['ts_code'].shift(1),'now_adj']=alldata.now_adj.shift(1) # 最新复权因子

alldata['close_qfq'] = alldata['close'] * alldata['adj_factor'] / alldata.now_adj  # 前复权 当日收盘价 × 当日复权因子 / 最新复权因子
alldata.to_csv('C:/quant/index/2020/test.csv') 
alldata.shape
alldata

,Unnamed: 0,open,high,low,close,pre_close,change,pct_chg,vol,amount,...,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount,ts_code,trade_date_x,adj_factor,now_adj,close_qfq
0,2935,16.65,16.95,16.55,16.87,16.45,0.42,2.5532,1530231.87,2571196.482,...,101102.47,606070,101705.05,143254,24246.44,000001.SZ,20200102,109.169,111.048,16.584549
1,49,16.94,17.31,16.92,17.18,16.87,0.31,1.8376,1116194.81,1914495.474,...,84137.06,301220,51703.29,209152,36004.28,000001.SZ,20200103,109.169,111.048,16.889304
2,676,17.01,17.34,16.91,17.07,17.18,-0.11,-0.6403,862083.50,1477930.193,...,43020.67,243151,41713.52,-66447,-11242.28,000001.SZ,20200106,109.169,111.048,16.781165
3,583,17.13,17.28,16.95,17.15,17.07,0.08,0.4687,728607.56,1247047.135,...,23499.73,165043,28243.25,13185,2360.01,000001.SZ,20200107,109.169,111.048,16.859812
4,2738,17.00,17.05,16.63,16.66,17.15,-0.49,-2.8571,847824.12,1423608.811,...,36629.78,277295,46547.42,-297399,-49932.23,000001.SZ,20200108,109.169,111.048,16.378103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762527,3800,57.60,58.18,57.13,57.22,57.67,-0.45,-0.7803,247572.87,1425408.390,...,30600.74,45230,26032.68,-8250,-4679.71,688981.SH,20201023,1.000,1.000,57.220000
762528,1348,56.89,58.80,56.26,58.45,57.22,1.23,2.1496,322355.87,1871698.729,...,34338.13,50715,29431.82,64816,37819.53,688981.SH,20201026,1.000,1.000,58.450000
762529,2462,58.00,59.02,57.50,58.70,58.45,0.25,0.4277,290649.23,1698753.437,...,26955.05,65695,38389.14,15890,9525.46,688981.SH,20201027,1.000,1.000,58.700000
762530,2002,58.55,59.59,57.81,59.33,58.70,0.63,1.0733,330595.53,1947820.833,...,31065.54,61195,36028.92,32511,19398.42,688981.SH,20201028,1.000,1.000,59.330000


In [77]:
from scipy.stats import rankdata
df=alldata[alldata['ts_code'].notna()]

close=df.close_qfq
diff, dea, my_macd = talib.MACD(close,
                                             fastperiod=12,
                                             slowperiod=26,
#获取均线的数据，通过timeperiod参数来分别获取 5,10,20 日均线的数据。
                                             signalperiod=9)
SMA5=talib.SMA(close,timeperiod=5)
SMA10=talib.SMA(close,timeperiod=10)
SMA20=talib.SMA(close,timeperiod=20)
SMA60=talib.SMA(close,timeperiod=60)
SMA120=talib.SMA(close,timeperiod=120)

EMA5=talib.EMA(close,timeperiod=5)
EMA10=talib.EMA(close,timeperiod=10)
EMA20=talib.EMA(close,timeperiod=20)
EMA60=talib.EMA(close,timeperiod=60)
EMA120=talib.EMA(close,timeperiod=120)

df['diff']=diff
df['dea']=dea
df['MACD']=my_macd
df['SMA5']=SMA5
df['SMA10']=SMA10
df['SMA20']=SMA20
df['SMA60']=SMA60
df['SMA120']=SMA120

df['EMA5']=EMA5
df['EMA10']=EMA10
df['EMA60']=EMA60
df['EMA120']=EMA120

df['EMA20']=EMA20
df['SMACD-K']=(diff-dea)/dea
df['C/S']=(close/SMA20-1)*100
df['M/L']=(SMA60/SMA120-1)*100
df['S/M']=(SMA20/SMA60-1)*100
df.reset_index(drop=True, inplace=True)

df.loc[df['ts_code'] == df['ts_code'].shift(4),'Chg5D']= (df.close_qfq/df.close_qfq.shift(4)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(9),'Chg10D']= (df.close_qfq/df.close_qfq.shift(9)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(19),'Chg20D']= (df.close_qfq/df.close_qfq.shift(19)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(59),'Chg60D']= (df.close_qfq/df.close_qfq.shift(59)-1)*100
df.loc[df['ts_code'] == df['ts_code'].shift(119),'Chg120D']= (df.close_qfq/df.close_qfq.shift(119)-1)*100





df['ATR'] = talib.ATR(df.high.values,df.low.values,df.pre_close.values, timeperiod=20)


df.fillna(0,inplace=True) #去除null
df.loc[(df['close_qfq']>df['EMA20']) & (df['close_qfq']>df['SMA20']) & (df['SMA20']>df['SMA20'].shift(19)),'kl1']=1 #破线拐头
df.loc[(df['EMA20']>df['EMA60']) & (df['close_qfq']>df['EMA20'].shift(19)),'K2']=1 #双均线交叉
df.loc[(df['EMA20']>df['SMA60']) & (df['close_qfq']>df['EMA20'].shift(19)),'K3']=1 
df.loc[(df['SMA20']>df['EMA60']) & (df['close_qfq']>df['SMA20'].shift(19)),'K4']=1 
df.loc[(df['SMA20']>df['SMA60']) & (df['close_qfq']>df['SMA20'].shift(19)),'K5']=1 

df.loc[(df['close_qfq']<df['EMA20']) & (df['close_qfq']<df['SMA20']) & (df['SMA20']<df['SMA20'].shift(19)),'kl1']=-1 #空头方向破线拐头
df.loc[(df['EMA20']<df['EMA60']) & (df['close_qfq']<df['EMA20'].shift(19)),'K2']=-1 #双均线交叉
df.loc[(df['EMA20']<df['SMA60']) & (df['close_qfq']<df['EMA20'].shift(19)),'K3']=-1 
df.loc[(df['SMA20']<df['EMA60']) & (df['close_qfq']<df['SMA20'].shift(19)),'K4']=-1 
df.loc[(df['SMA20']<df['SMA60']) & (df['close_qfq']<df['SMA20'].shift(19)),'K5']=-1 
df.fillna(0,inplace=True) #去除null
df['trendscore-L']=(df.kl1+df.K2+df.K3+df.K4+df.K5)

df.loc[(df['close_qfq']>df['EMA10']) & (df['close_qfq']>df['SMA10']) & (df['SMA10']>df['SMA10'].shift(19)),'ks1']=1 #破线拐头
df.loc[(df['EMA10']>df['EMA20']) & (df['close_qfq']>df['EMA10'].shift(19)),'KK2']=1 #双均线交叉
df.loc[(df['EMA10']>df['SMA20']) & (df['close_qfq']>df['EMA10'].shift(19)),'KK3']=1 
df.loc[(df['SMA10']>df['EMA20']) & (df['close_qfq']>df['SMA10'].shift(19)),'KK4']=1 
df.loc[(df['SMA10']>df['SMA20']) & (df['close_qfq']>df['SMA10'].shift(19)),'KK5']=1 

df.loc[(df['close_qfq']<df['EMA10']) & (df['close_qfq']<df['SMA10']) & (df['SMA10']<df['SMA10'].shift(19)),'ks1']=-1 #空头方向破线拐头
df.loc[(df['EMA10']<df['EMA20']) & (df['close_qfq']<df['EMA10'].shift(19)),'KK2']=-1 #双均线交叉
df.loc[(df['EMA10']<df['SMA20']) & (df['close_qfq']<df['EMA10'].shift(19)),'KK3']=-1 
df.loc[(df['SMA10']<df['EMA20']) & (df['close_qfq']<df['SMA10'].shift(19)),'KK4']=-1 
df.loc[(df['SMA10']<df['SMA20']) & (df['close_qfq']<df['SMA10'].shift(19)),'KK5']=-1 
df.fillna(0,inplace=True) #去除null
df['trendscore-S']=(df.ks1+df.KK2+df.KK3+df.KK4+df.KK5)
df = pd.merge(df, ISindex, how='left',on='ts_code') 

df=df[['trade_date_x','ts_code','close_qfq','amount','pct_chg','ATR','Chg5D','Chg10D','Chg20D','Chg60D','Chg120D','C/S','S/M','M/L','ks1','kl1','trendscore-S','trendscore-L']]

df.head(10)


,trade_date_x,ts_code,close_qfq,amount,pct_chg,ATR,Chg5D,Chg10D,Chg20D,Chg60D,Chg120D,C/S,S/M,M/L,ks1,kl1,trendscore-S,trendscore-L
0,20200102,000001.SZ,16.584549,2571196.482,2.5532,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20200103,000001.SZ,16.889304,1914495.474,1.8376,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20200106,000001.SZ,16.781165,1477930.193,-0.6403,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20200107,000001.SZ,16.859812,1247047.135,0.4687,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20200108,000001.SZ,16.378103,1423608.811,-2.8571,0.0,-1.244813,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,20200109,000001.SZ,16.505903,1725326.806,0.7803,0.0,-2.270081,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,20200110,000001.SZ,16.407595,975154.818,-0.5956,0.0,-2.226128,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,20200113,000001.SZ,16.702519,1468271.683,1.7975,0.0,-0.932945,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,20200114,000001.SZ,16.476411,2217608.852,-1.3537,0.0,0.600240,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,20200115,000001.SZ,16.240472,1424889.228,-1.4320,0.0,-1.608100,-2.074689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
latestday=max(df.trade_date_x)
valve=150000   #以1亿5000万元为界限，150000K
df2=df[(df.trade_date_x==latestday) & (df.amount > valve)]
df2=pd.merge(df2,pool,how='left',on='ts_code')
df2=df2.drop(df2.columns[-2:-1],axis=1)
df2=pd.merge(df2,ISindex2,how='left',on='ts_code')
df2=df2.drop(["symbol",'area','list_date','index_code'],axis=1)
df2.rename(columns={'industry_name':'industry_L1','industry':'industry_L2'}, inplace = True)
order = ['trade_date_x', 'ts_code', 'name', 'industry_L1', 'industry_L2', 'K1', 'Indextrendscore-L', 'close_qfq','amount','pct_chg','ATR','Chg5D','Chg10D','Chg20D','Chg60D','Chg120D','C/S','S/M','M/L','ks1','kl1','trendscore-S','trendscore-L']
df2 = df2[order]
df2


,trade_date_x,ts_code,name,industry_L1,industry_L2,K1,Indextrendscore-L,close_qfq,amount,pct_chg,...,Chg20D,Chg60D,Chg120D,C/S,S/M,M/L,ks1,kl1,trendscore-S,trendscore-L
0,20201029,000001.SZ,平安银行,银行,银行,1.0,5.0,17.77,1498040.947,0.7941,...,17.526455,32.909499,29.762057,7.080446,7.954853,6.204732,1.0,1.0,5.0,5.0
1,20201029,000002.SZ,万科A,房地产,全国地产,-1.0,-5.0,27.38,1703222.368,1.7088,...,-2.562278,5.627403,5.942709,-1.615193,0.707739,3.204362,-1.0,-1.0,-5.0,-1.0
2,20201029,000004.SZ,国农科技,医药生物,互联网,0.0,2.0,27.17,185073.414,2.9167,...,18.594500,-4.933520,-1.842486,9.866559,-2.800432,-7.740703,1.0,1.0,5.0,1.0
3,20201029,000009.SZ,中国宝安,综合,综合类,0.0,-4.0,7.73,424440.113,-1.6539,...,12.682216,-6.860621,18.127992,2.683316,-1.907582,-2.189139,1.0,1.0,5.0,1.0
4,20201029,000012.SZ,南玻A,建筑材料,玻璃,-1.0,-5.0,6.12,295114.928,2.8571,...,14.606742,5.699482,38.846797,4.108191,-0.110450,8.541536,1.0,1.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,20201029,688536.SH,思瑞浦,NaN,半导体,NaN,NaN,283.50,283947.462,4.8486,...,7.386364,0.000000,0.000000,1.713341,135.830142,59.917192,0.0,1.0,3.0,5.0
1227,20201029,688567.SH,孚能科技,NaN,电气设备,NaN,NaN,31.86,261751.328,0.5047,...,19.729425,17.304860,0.000000,7.294403,7.577862,-10.486824,1.0,1.0,5.0,5.0
1228,20201029,688580.SH,伟思医疗,NaN,医疗保健,NaN,NaN,126.40,463728.144,-19.4904,...,-27.008142,-31.675676,0.000000,-24.062648,-7.427624,65.202152,-1.0,-1.0,-5.0,-3.0
1229,20201029,688788.SH,科思科技,NaN,通信设备,NaN,NaN,187.88,594677.867,2.6611,...,0.000000,0.000000,0.000000,155.030915,79.456293,28.737525,1.0,1.0,5.0,5.0


In [79]:
#找长期，中期乖离率高，但短期下杀严重，趋势却走好的
DF=df2[(df2['C/S']<0) & (df2['S/M']>5) & (df2['M/L']> 5) & (df2['trendscore-S']>=3) & (df2['ks1']==1)]
DF

,trade_date_x,ts_code,name,industry_L1,industry_L2,K1,Indextrendscore-L,close_qfq,amount,pct_chg,...,Chg20D,Chg60D,Chg120D,C/S,S/M,M/L,ks1,kl1,trendscore-S,trendscore-L


In [83]:
#writer = pd.ExcelWriter(r'C:/quant/output/combinedoutput.csv')
df2.to_csv(r'C:/quant/output/dailyoutput.csv',index=False,encoding='utf-8-sig')
#这里“AllData”是sheet的名字
#writer.save()#执行完这一步之后，合并后的表格就保存在了C:\Users\output.xls中

In [ ]:
#美债利率

#获取5年期和20年期数据
Udf = pro.us_tltr(start_date='20180101', end_date='20200907', fields='ltc,cmt')
Udf.to_csv("C:/quant/index/UDF.csv", index=False)


In [ ]:
#可转债
EEf = pro.cb_daily(trade_date='20200821')

EFF= pro.cb_basic(fields="ts_code,bond_short_name,stk_code,stk_short_name,list_date,delist_date")   

    


EFF.to_csv(r'C:/quant/debt/2020/20200821.csv',index=False,encoding='utf-8-sig')
 

    


In [65]:

#取000001的前复权行情
EEf = ts.pro_bar(ts_code='000001.SZ', adj='qfq', start_date='20200101', end_date='20201029')
EEf.to_csv(r'C:/quant/output/testoutput.csv',index=False,encoding='utf-8-sig')
